# 1. はじめに

このCodeは、[リンクアンドモチベーション Advent Calendar 2024](https://qiita.com/advent-calendar/2024/lmi)の24日目の記事です。
私は現在ソフトウェアエンジニアとして働いていますが、2024年は個人的にデータサイエンスまわりに興味を持った年でした。
その中で社内で週1程度行っているデータサイエンス関連の勉強会に参加し、データサイエンティストの方と一緒に[［第3版］Python機械学習プログラミング 達人データサイエンティストによる理論と実践](https://book.impress.co.jp/books/1120101017)を読む機会がありました。
本で紹介されていた、SVMを使って非線形問題を解くことが面白いと感じたので、題材にして記事を書きます。具体的には、2次元上で線形分類できないデータセットに対し、カーネルSVMの基本的な考え方である**高次元へ射影して線形分離できるようにする**を可視化することで理解を深めることを目的とします。

# 2. 準備

基本的なライブラリをインポートしておきます。

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 3. 使用するデータセット

上記書籍で題材として作成されているのは、NumPyの`logical_xor`関数（排他的論理和）を用いたデータセットですが、ここでは（どうせなので）別のデータとして、[scikit-learnのmake_moons](https://scikit-learn.org/dev/modules/generated/sklearn.datasets.make_moons.html) を用います。
このデータは、半月状に分布する2次元のデータであり、直線では分離できない構造を持ちます。

In [ ]:
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=100, noise=0.1)

次のように散布図を描画すると、make_moons感が出ます🌛。
ひとまず上記では、`n_samples`や`noise`は適当に設定しましたが、次の記事で各パラメータに対する影響をわかりやすく紹介してくださっています：[[scikit-learn] 4. make_moonsによる三日月状データの生成](https://sabopy.com/py/scikit-learn-4/)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.coolwarm)
plt.title("Nonlinear Data (2D)")
plt.show()

# 4. 2次元平面での非線形分離

非線形なデータを分類するため、モデルを構築し、学習します。ここでは、RBFカーネルSVMを用いています。

In [ ]:
from sklearn.svm import SVC

model = SVC(kernel='rbf', C=1.0, gamma=0.5)
model.fit(X, y)

モデルに予測するため、グリッドを生成します。

In [ ]:
xx, yy = np.meshgrid(np.linspace(-1.5, 2.5, 200), np.linspace(-1, 1.5, 200))

モデルによる予測をします。

In [ ]:
Z = model.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

決定境界を含めて、描画します。

In [ ]:
plt.contourf(xx, yy, Z, alpha=0.75, cmap=plt.cm.coolwarm)
plt.scatter(X[:, 0], X[:, 1], c=y, edgecolors='k')
plt.title("Decision Boundary with RBF Kernel")
plt.show()

# 5. 3次元空間への射影

RBFカーネルに基づく特徴量の生成をします。

In [ ]:
def rbf_projection(X, gamma=0.5):
    return np.exp(-gamma * np.linalg.norm(X, axis=1)**2)

新しい次元を追加し、新しい特徴量を生成します。

In [ ]:
Z = rbf_projection(X)
X_projected = np.hstack((X, Z.reshape(-1, 1)))

諸々3次元にプロットします。

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X_projected[:, 0], X_projected[:, 1], X_projected[:, 2], c=y, cmap=plt.cm.coolwarm)
ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('RBF Feature')
plt.title("3D Projection of the Data")
plt.show()

# 6. 3次元空間での線形分離

こちらも同様にプロットする。

In [ ]:
xx, yy = np.meshgrid(np.linspace(X_projected[:, 0].min(), X_projected[:, 0].max(), 50),
                     np.linspace(X_projected[:, 1].min(), X_projected[:, 1].max(), 50))
zz = np.full(xx.shape, 0.5)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(X_projected[:, 0], X_projected[:, 1], X_projected[:, 2], c=y, cmap=plt.cm.coolwarm)
ax.plot_surface(xx, yy, zz, alpha=0.3, color='gray')

ax.set_xlabel('X1')
ax.set_ylabel('X2')
ax.set_zlabel('RBF Feature')
plt.title("3D Projection with Linear Separating Plane")
plt.show()